# Sesiones prácticas

## 1

Instalación, python, numpy

## 2

Dispositivos de captura


- webcam.py con opencv crudo
- spyder
- umucv (install con --upgrade)
- PYTHONPATH para sus módulos
- stream.py, opciones de autostream, efecto de teclas
- ejemplo de B/N invertido
- grabar video de demo
- servidor mjpeg,

## 3

- medidor.py
- inrange.py

## 4

- captura en hilo aparte
- mean shift / camshift

## 5

- hog/hog0.py

- hog/pedestrian.py con

- dlib: hog/facelandmarks.py

- dlib: herramienta imglab, hog/train_detector.py, hog/run_detector.py

## 6

- LK/corners0.py, 1, 2, 3, LK/lk_tracks.py

## 7

En esta sesión vamos a experimentar con el detector de puntos de interés SIFT. (La implementación de opencv está en un repositorio aparte con las contribuciones "non free", pero la patente ha expirado hace unos días. En cualquier caso, la versión de opencv que estamos usando lo incluye.)

Nuestro objetivo es calcular un conjunto de "keypoints", cada uno con su descriptor (vector de características que describe el entorno del punto), que nos permita encontrarlo en imágenes futuras. Esto tiene una aplicación inmediata para reconocer objetos y más adelante en geometría visual.

Empezamos con el ejemplo de código code/SIFT/sift0.py, que simplemente calcula y muestra los puntos de interés. Es interesante observar el efecto de los parámetros del método y el tiempo de cómputo en función del tamaño de la imagen (que puedes cambiar con --size o --resize).

El siguiente ejemplo code/SIFT/sift1.py muestra un primer ataque para establecer correspondencias. Los resultados son bastante pobres.

Finalmente, en code/SIFT/sift.py aplicamos un criterio de selección para eliminar muchas correspondencias erróneas (aunque no todas). Esto es en principio suficiente para el reconocimiento de objetos. (Más adelante veremos una forma mucho mejor de eliminar correspondencias erróneas, necesaria para aplicaciones de geometría.)

El ejercicio obligatorio **SIFT** es una ampliación sencilla de este código. Se trata de almacenar un conjunto de modelos (¡con textura! para que tengan suficientes keypoints) como portadas de libros, discos, videojuegos, etc. y reconocerlos en base a la proporción de coincidencias detectadas.

Una segunda actividad en esta sesión consiste en comentar el ejemplo de código code/server.py. Utiliza el paquete [flask][flask] para crear un sencillo servidor web que devuelve la imagen de la cámara modificada como deseemos. Sirve como punto de partida para el ejercicio opcional **WEB**.

[flask]: https://en.wikipedia.org/wiki/Flask_(web_framework)

## 8

En esta sesión vamos a explorar el reconocimiento de formas mediante descriptores frecuenciales.

Nuestro objetivo es hacer un programa que reconozca la forma de trébol, como se muestra [en este pantallazo](../../images/demos/shapedetect.png). Si no tenéis a mano un juego de cartas podéis usar --dev=dir:../images/card*.png para hacer las pruebas, aunque lo ideal es hacerlo funcionar con una cámara en vivo.

Trabajaremos con los ejemplos de la carpeta `code/shapes` y, como es habitual, iremos añadiendo poco a poco funcionalidad. En cada nuevo paso los comentarios explican los cambios respecto al paso anterior.

Empezamos con el ejemplo shapes/trebol1.py, que simplemente prepara un bucle de captura básico, binariza la imagen y muestra los contornos encontrados. Se muestran varias formas de realizar la binarización y se puede experimentar con ellas, pero en principio el método automático propuesto suele funcionar bien en muchos casos.

El segundo paso en shapes/trebol2.py junta la visualización en una ventana y selecciona los contornos oscuros de tamaño razonable. Esto no es imprescincible para nuestra aplicación, pero es interesante trabajar con el concepto de orientación de un contorno.

En shapes/trebol3.py leemos un modelo de la silueta trébol de una imagen que tenemos en el repositorio y la mostramos en una ventana.

En shapes/trebol3b.py hacemos una utilidad para ver gráficamente las componentes frecuenciales como elipses que componen la figura. Podemos ver las componentes en su tamaño natural, incluyendo la frecuencia principal, [como aquí](../images/demos/full-components.png), o quitando la frecuencia principal y ampliando el tamaño de las siguientes, que son la base del descriptor de forma, [como se ve aquí](../images/demos/shape-components.png). Observa que las configuraciones de elipses son parecidas cuando corresponden a la misma silueta.

En shapes/trebol4.py definimos la función que calcula el descriptor invariante. Se basa esencialmente en calcular los tamaños relativos de estas elipses. En el código se explica cómo se consigue la invarianza a las transformaciones deseadas: posición, tamaño, giros, punto de partida del contorno y ruido de medida.

Finalmente, en shapes/trebol5.py calculamos el descriptor del modelo y en el bucle de captura calculamos los descriptores de los contornos oscuros detectados para marcar las siluetas que tienen un descriptor muy parecido al del trébol.

El ejercicio opcional SILU consiste en ampliar este código para reconocer un conjunto más amplio de siluetas en alguna aplicación que se os parezca interesante. Por ejemplo, en images/shapes tenéis los modelos de caracteres de las placas de matrícula.

## 9

En esta sesión vamos a hacer varias actividades. Necesitamos algunos paquetes. En Linux son:

    sudo apt install tesseract-ocr tesseract-ocr-spa libtesseract-dev
    pip install tesserocr

    sudo apt install libzbar-dev
    pip install pyzbar

Usuarios de Mac y Windows: investigad la forma de instalarlo.

1) En primer lugar nos fijamos en el script `code/ocr.png`, cuya misión es poner en marcha el OCR con la cámara en vivo. Usamos el paquete de python `tesserocr`. Vamos a verificar el funcionamiento con una imagen estática, pero lo ideal es probarlo con la cámara en vivo.

    python ocr.py python ocr.py --dev=dir:../images/texto/bo0.png 

Está pensado para marcar una sola línea de texto, [como se muestra aquí](../images/demos/ocr.png). Este pantallazo se ha hecho con la imagen bo1.png disponible en la misma carpeta, que está desenfocada, pero aún así el OCR funciona bien.


2) El segundo ejemplo es `code/zbardemo.png` que muestra el uso del paquete pyzbar para leer códigos de barras ([ejemplo](../images/demos/barcode.png)) y códigos QR ([ejemplo](../images/demos/qr.png)) con la cámara. En los códigos de barras se detectan puntos de referencia, y en los QR se detectan las 4 esquinas del cuadrado, que pueden ser útiles como referencia en algunas aplicaciones de geometría.


3) A continuación vamos a jugar con un bot de telegram que nos permite comunicarnos cómodamente con nuestro ordenador desde el teléfono móvil, sin necesidad de tener una dirección pública de internet.

Voy a dejar durante esta mañana un bot funcionando para que hagáis pruebas. El bot se llama "BichoBot" y su foto  de perfil es una pequeña plataforma con ruedas con un raspberry pi encima. Responde al comando /hello y si le enviáis una foto os la devolverá en blanco y negro e invertida. (Está basado en bot3.py).

Simplemente necesitamos:

    pip install python-telegram-bot

El ejemplo `bot/bot0.py` nos envía al teléfono la IP del ordenador (es útil si necesitamos conectarnos por ssh con una máquina que tiene IP dinámica).

El ejemplo `bot/bot1.py` explica la forma de enviar una imagen nuestro teléfono cuando ocurre algo. En este caso se envía cuando se pulsa una tecla, pero lo normal es detectar automáticamente algún evento con las técnicas de visión artificial que estamos estudiando.

El ejemplo `bot/bot2.py` explica la forma de hacer que el bot responda a comandos. El comando /hello nos devuelve el saludo, el comando /stop detiene el programa y el comando /image nos devuelve una captura de nuestra webcam. (Se ha usado la captura en un hilo). 

El ejemplo `bot/bot3.py` explica la forma de capturar comandos con argumentos y el procesamiento de una imagen enviada por el usuario.

Esta práctica es completamente opcional, pero es muy útil para enviar cómodamente a nuestros programas de visión artificial una imagen tomada con la cámara sin necesidad de escribir una aplicación específica para el móvil. Algunos ejercicios que estamos haciendo se pueden adaptar fácilmente para probarlos a través de un bot de este tipo.

Para crearos vuestro propio bot tenéis que contactar con el bot de telegram "BotFather", que os guiará paso a paso y os dará el token de acceso. Y luego el "IDBot" os dirá el id numérico de vuestro usuario.

En la carpeta hay otros ejemplos más avanzados.


4) En la dirección 

https://github.com/ruvelro/TV-Online-TDT-Spain

se pueden encontrar las url de muchos canales de televisión que están haciendo streaming en directo. Abriendo los ficheros m3u8 encontramos las url que podemos poner en --dev en nuestras aplicaciones (hay distintas resoluciones de imagen). Por ejemplo, la TVE1 está aquí:

http://hlsliveamdgl7-lh.akamaihd.net/i/hlsdvrlive_1@583042/index_0400_av-p.m3u8?sd=10&rebase=on

(Funciona a saltos porque autoStream lee los frames lo más rápido posible. Se puede poner un time.sleep para que vaya a ritmo normal).

Próximamente propondré un ejercicio opcional relacionado con esto.

## 10

Esta sesión está dedicada a poner en marcha una red convolucional sencilla. La tarea que vamos a resolver es el reconocimiento de dígitos manuscritos. Por eso, en primer lugar es conveniente escribir unos cuantos números en una hoja de papel, con un bolígrafo que tenga un trazo no demasiado fino, y sin preocuparnos mucho de que estén bien escritos. Pueden tener distintos tamaños, pero no deben estar muy girados. Para desarrollar el programa y hacer pruebas cómodamente se puede trabajar con una imagen fija, pero la idea es que nuestro programa funcione con la cámara en vivo.


Trabajaremos en la carpeta [code/DL/CNN](../code/DL/CNN), donde tenemos las diferentes etapas de ejercicio y una imagen de prueba.

El primer paso es `digitslive-1.py` que simplemente encuentra las manchas de tinta que pueden ser posibles números.

En `digitslive-2.py` normalizamos el tamaño de las detecciones para poder utilizar la base de datos MNIST.

En `digitslive-3.py` implementamos un clasificador gaussiano con reducción de dimensión mediante PCA y lo ponemos en marcha con la imagen en vivo. (Funciona bastante bien pero, p.ej., en la imagen de prueba comete un error).

Finalmente, en `digitslive-4.py` implementamos la clasificación mediante una red convolucional mediante el paquete **keras**. Usamos unos pesos precalculados. (Esta máquina ya no comete el error anterior.)

Como siempre, en cada fase del ejercicio los comentarios explican el código que se va añadiendo.

Una vez conseguido esto, la sesión práctica tiene una segunda actividad que consiste en **entrenar los pesos** de (por ejemplo) esta misma red convolucional. Para hacerlo en nuestro ordenador sin perder la paciencia necesitamos una GPU con CUDA y libCUDDN. La instalación de todo lo necesario puede no ser trivial. 

Una alternativa muy práctica es usar [google colab](https://colab.research.google.com/), que proporciona gratuitamente máquinas virtuales con GPU y un entorno de notebooks jupyter (un poco modificados pero compatibles). Para probarlo, entrad con vuestra cuenta de google y abrid un nuevo notebook. En la opción de menú **Runtime** hay que seleccionar **Change runtime type** y en hardware accelerator ponéis GPU. En una celda del notebook copiáis directamente el contenido del archivo `cnntest.py` que hay en este mismo directorio donde estamos trabajando hoy. Al evaluar la celda se descargará la base de datos y se lanzará un proceso de entrenamiento. Cada epoch tarda unos 4s. Podéis comparar con lo que se consigue con la CPU en vuestro propio ordenador. Se puede lanzar un entrenamiento más completo, guardar los pesos y descargarlos a vuestra máquina.

Como curiosidad, podéis comparar con lo que conseguiría el OCR tesseract, y guardar algunos casos de dígitos que estén bien dibujados pero que la red clasifique mal.